In [ ]:
# basic cherry-pick

from adaptyv_lab import Protocol
import pandas as pd
from rich import print

lay_file = "C:\\Users\\Adaptyvbio\\dev\\hamilton-protocols\\src\\hamilton_protocols\\layouts\\cherry-pick.lay"
cherry_pick_file = "C:\\Users\\Adaptyvbio\\Documents\\MAPS\\CHP-005.csv"

mapping = pd.read_csv(cherry_pick_file)
src_plate_names = mapping["Source Plate"].unique()
protocol = Protocol.from_layout("test", lay_file, simulator_mode=False)

if protocol.deck is None:
    raise ValueError("No deck found in the layout file.")
print(protocol.deck.grid)

tip_rack = protocol.deck.get_tip_rack("E1")
src_plates = protocol.deck.get_plate_stack("F4")[: len(src_plate_names)]
dst_plates = protocol.deck.get_plate_stack("F3")[: len(src_plate_names)][::-1]
src_plate = protocol.deck.get_plate("C3")
dst_plate = protocol.deck.get_plate("C2")

tips = tip_rack.iterlabware(num_channels=1)

protocol.initialize()

for src_plate_name in src_plate_names:
    protocol.grip_get(src_plates.pop())
    protocol.grip_place(src_plate, eject_tool=1)

    print(f"Processing source plate: {src_plate_name}")

    src_wells = mapping[mapping["Source Plate"] == src_plate_name][
        "Source Well"
    ].to_list()
    dst_wells = mapping[mapping["Source Plate"] == src_plate_name][
        "Destination Well"
    ].to_list()
    well_map = {
        src_well: dest_well for src_well, dest_well in zip(src_wells, dst_wells)
    }
    for group in src_plate[src_wells].itergroups(num_channels=1):
        group_src_wells = [pos.alphanumeric for pos in group]
        group_dst_wells = [well_map[pos.alphanumeric] for pos in group]
        tips_to_use = next(tips)
        protocol.pickup_tips(tips_to_use)
        protocol.aspirate(src_plate[group_src_wells], 20).dispense(
            dst_plate[group_dst_wells], 20
        )
        print(
            src_plate[group_src_wells].positions_str,
            "->",
            dst_plate[group_dst_wells].positions_str,
        )
        protocol.eject_tips(tips_to_use)

    protocol.grip_get(src_plate)
    protocol.grip_place(dst_plates.pop())

protocol.grip_eject()


async for response in protocol.run():
    print(response)

In [ ]:
# fab remapping
from hamilton_protocols.protocols.dna import (
    fab_mapping_protocol,
    FAbMappingParams,
    FAbMappingProtocolParams,
)
from hamilton_protocols.utils import dataframe_to_base64
import pandas as pd

df_b64 = dataframe_to_base64(
    pd.read_csv("C:\\Users\\Adaptyvbio\\Documents\\MAPS\\CRD-002-012_fabs.csv")
)

protocol = fab_mapping_protocol(
    FAbMappingProtocolParams(plates=[FAbMappingParams(plate_id="11", csv_data=df_b64)])
)

async for response in protocol.run():
    pass

In [ ]:
# dna reconstitution
from hamilton_protocols.protocols.dna import (
    DNAReconstitutionParams,
    DNAPlateParams,
    dna_reconstitution_protocol,
)
from src.hamilton_protocols.utils import dataframe_to_base64
import pandas as pd

plates = [191, 187, 190, 188]

plate_data = {}
for plate in plates:
    plate_data[plate] = dataframe_to_base64(
        pd.read_csv(
            f"C:\\Users\\Adaptyvbio\\Documents\\MAPS\\P-TWI-0{plate}_hamilton_reconstitution_mappings.csv"
        )
    )


protocol = dna_reconstitution_protocol(
    DNAReconstitutionParams(
        plates=[
            DNAPlateParams(plate_id=f"{plate}", csv_data=data)
            for plate, data in plate_data.items()
        ]
    ),
    simulate=False,
)

async for response in protocol.run():
    print(response)

In [ ]:
# SPR

from adaptyv_lab import Protocol

protocol = Protocol.from_layout(
    "test",
    "C:\\Users\\Adaptyvbio\\dev\\hamilton-protocols\\src\\hamilton_protocols\\layouts\\bli.lay",
    simulator_mode=False,
)
if protocol.deck is None:
    raise ValueError("No deck found in the layout file.")

plate = protocol.deck.get_plate("E5")
reservoir = protocol.deck.get_reservoir("B3")
tips = protocol.deck.get_tip_rack("A3")
src_plate_stack = protocol.deck.get_plate_stack("F4")[:4]
dst_plate_stack = protocol.deck.get_plate_stack("F3")[:4][::-1]

protocol.initialize()

# add buffer
for _ in range(4):
    protocol.pickup_tips(tips)
    protocol.grip_get(src_plate_stack.pop()).grip_place(plate)
    for q in [(0, 1), (1, 0), (1, 1)]:
        for _ in range(2):
            protocol.aspirate(reservoir, 40)
            protocol.dispense(plate[q], 40, liquid_height=2)
    protocol.eject_tips(mode=1)
    protocol.pickup_tips(tips[::2, -20::2])
    protocol.aspirate(reservoir, 40)
    protocol.dispense(plate[0, 0], 40, liquid_height=2)
    protocol.eject_tips(mode=1)
    protocol.grip_get(plate).grip_place(dst_plate_stack.pop())

# # add samples
# for q in [(0, 0), (1, 0), (0, 1), (1, 1)]:
#     protocol.grip_get(tip_stack.pop()).grip_place(tips)
#     protocol.grip_get(sample_plate_stack_src.pop()).grip_place(sample_plate)

#     protocol.pickup_tips(tips)
#     protocol.aspirate(sample_plate, 8)
#     protocol.dispense(plate[q], 8, liquid_height=15, mix_cycles=2, mix_volume=10)
#     protocol.eject_tips(mode=2)

#     protocol.grip_get(sample_plate).grip_place(sample_plate_stack_dst.pop())
#     protocol.grip_get(tips).grip_place(tips, waste=True)

protocol.grip_eject()

async for response in protocol.run():
    print(response)

In [ ]:
# dna dilution

from hamilton_protocols.protocols.dna import (
    DNADilutionProtocolParams,
    dna_dilution_protocol,
    DNADilutionSourcePlateParams,
    DNADilutionDestinationPlateParams,
)

protocol = dna_dilution_protocol(
    DNADilutionProtocolParams(
        plates=[
            DNADilutionDestinationPlateParams(
                plate_id="P-DIL-0153",
                source_plates=[
                    DNADilutionSourcePlateParams(
                        plate_id="P-REC-0105",
                        rows=8,
                        cols=6,
                    ),
                ],
            ),
            DNADilutionDestinationPlateParams(
                plate_id="P-DIL-0154",
                source_plates=[
                    DNADilutionSourcePlateParams(
                        plate_id="P-REC-0105",
                        rows=8,
                        cols=3,
                        source_well="A7",
                    ),
                    DNADilutionSourcePlateParams(
                        plate_id="P-REC-0106",
                        rows=3,
                        cols=1,
                    ),
                ],
            ),
        ]
    ),
    simulate=True,
)

In [8]:
# expression utils
lay_file = "C:\\Users\\Adaptyvbio\\dev\\hamilton-protocols\\src\\hamilton_protocols\\layouts\\reconstitute.lay"

protocol = Protocol.from_layout("test", lay_file, simulator_mode=False)

if protocol.deck is None:
    raise ValueError("No deck found in the layout file.")
print(protocol.deck.grid)

tip_rack = protocol.deck.get_tip_rack("D2")
tip_stack = protocol.deck.get_tip_rack_stack("E3")[:2]
exp_plates_src = protocol.deck.get_plate_stack("F2")[:2]
exp_plates_dst = protocol.deck.get_plate_stack("F3")[:2][::-1]
dil_plates_src = protocol.deck.get_plate_stack("F1")[:2]
dil_plates_dst = protocol.deck.get_plate_stack("F4")[:2][::-1]
exp_plate = protocol.deck.get_plate("C2")
dil_plate = protocol.deck.get_plate("C3")

volumes = [1.5, 1]

protocol.initialize()
for vol in volumes:
    protocol.grip_get(exp_plates_src.pop()).grip_place(exp_plate)
    protocol.grip_get(dil_plates_src.pop()).grip_place(dil_plate)
    protocol.grip_get(tip_stack.pop()).grip_place(tip_rack)

    protocol.pickup_tips(tip_rack)
    protocol.aspirate(dil_plate, vol).dispense(
        exp_plate, vol, mix_cycles=3, mix_volume=2
    )
    protocol.eject_tips(mode=2)

    protocol.grip_get(exp_plate).grip_place(exp_plates_dst.pop())
    protocol.grip_get(dil_plate).grip_place(dil_plates_dst.pop())
    protocol.grip_get(tip_rack).grip_place(tip_rack, waste=True)
protocol.grip_eject()

async for response in protocol.run():
    print(response)

{
    'C': {'3': ['C3_plate96'], '4': ['C4_plate96'], '2': ['C2_plate96']},
    'F': {
        '2': ['F2_plate96_0001', 'F2_plate96_0002', 'F2_plate96_0003', 'F2_plate96_0004'],
        '1': ['F1_plate96_0001', 'F1_plate96_0002', 'F1_plate96_0003', 'F1_plate96_0004'],
        '3': ['F3_plate96_0001', 'F3_plate96_0002', 'F3_plate96_0003', 'F3_plate96_0004'],
        '4': ['F4_plate96_0001', 'F4_plate96_0002', 'F4_plate96_0003', 'F4_plate96_0004']
    },
    'E': {
        '1': ['E1_tip96_0001', 'E1_tip96_0002', 'E1_tip96_0003', 'E1_tip96_0004'],
        '3': ['E3_tip96in384_0001', 'E3_tip96in384_0002', 'E3_tip96in384_0003', 'E3_tip96in384_0004']
    },
    'B': {'5': ['B5_reservoir300']},
    'A': {'3': ['A3_tip96in384'], '4': ['A4_tip96in384']},
    'D': {'2': ['D2_tip96in384']}
}

2025-05-26 19:21:34,094 - adaptyv_lab.core.interface - INFO - Found existing GUI process, terminating it
2025-05-26 19:21:34,095 - adaptyv_lab.core.interface - INFO - Starting Hamilton client
2025-05-26 19:21:34,096 - adaptyv_lab.core.interface - INFO - Setting up HSL process in GUI mode
2025-05-26 19:21:34,096 - adaptyv_lab.core.interface - INFO - Updating HSL file to use layout: C:\Users\Adaptyvbio\dev\hamilton-protocols\src\hamilton_protocols\layouts\reconstitute.lay


2025-05-26 19:21:34,795 - adaptyv_lab.core.interface - INFO - Checking if HSL application is running
Timed out after 1.5 seconds waiting for response to command ping (id: ping_check)
Traceback (most recent call last):
  File "C:\Users\Adaptyvbio\AppData\Roaming\uv\python\cpython-3.13.3-windows-x86_64-none\Lib\asyncio\tasks.py", line 507, in wait_for
    return await fut
           ^^^^^^^^^
  File "C:\Users\Adaptyvbio\AppData\Roaming\uv\python\cpython-3.13.3-windows-x86_64-none\Lib\asyncio\queues.py", line 186, in get
    await getter
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\Adaptyvbio\dev\hamilton-protocols\.venv\Lib\site-packages\adaptyv_lab\core\client.py", line 213, in send_command
    response_data = await asyncio.wait_for(
                    ^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
    )
    ^
  File "C:\Users\Adaptyvbio\AppData\Roaming\uv\python\cpython-3.13.3-

HamiltonResponse(
    raw='{"command":"STAR-return","step-name":"initialize","step-return1":"0[01,00,00,0,,Waste,[02,00,00,0,,Waste,[
03,00,00,0,,Waste,[04,00,00,0,,Waste,","step-return2":"","step-return3":"","step-return4":"","id":"d20a96a9-4961-45
54-b20e-4508df35a888"}',
    status='SUCCESS',
    errors=[],
    data={
        'command': 'STAR-return',
        'step-name': 'initialize',
        'step-return1': '0[01,00,00,0,,Waste,[02,00,00,0,,Waste,[03,00,00,0,,Waste,[04,00,00,0,,Waste,',
        'step-return2': '',
        'step-return3': '',
        'step-return4': ''
    }
)

HamiltonResponse(
    raw='{"command":"STAR-return","step-name":"gripGet","step-return1":"0[01,00,00,0,,F2_plate96_0002,A1","step-ret
urn2":"","step-return3":"","step-return4":"","id":"fc3ac042-a763-4ced-8913-b8f80ca73b35"}',
    status='SUCCESS',
    errors=[],
    data={
        'command': 'STAR-return',
        'step-name': 'gripGet',
        'step-return1': '0[01,00,00,0,,F2_plate96_0002,A1',
        'step-return2': '',
        'step-return3': '',
        'step-return4': ''
    }
)

...

HamiltonResponse(
    raw='{"command":"STAR-return","step-name":"gripPlace","step-return1":"0[01,00,00,0,,F4_plate96_0002,A1","step-r
eturn2":"","step-return3":"","step-return4":"","id":"433b0fc6-8a12-4f63-b52c-ada8333214bd"}',
    status='SUCCESS',
    errors=[],
    data={
        'command': 'STAR-return',
        'step-name': 'gripPlace',
        'step-return1': '0[01,00,00,0,,F4_plate96_0002,A1',
        'step-return2': '',
        'step-return3': '',
        'step-return4': ''
    }
)

HamiltonResponse(
    raw='{"command":"STAR-return","step-name":"gripGet","step-return1":"0[01,00,00,0,,D2_tip96in384,1","step-return
2":"","step-return3":"","step-return4":"","id":"8408735b-265e-49c4-a93c-5da639e73363"}',
    status='SUCCESS',
    errors=[],
    data={
        'command': 'STAR-return',
        'step-name': 'gripGet',
        'step-return1': '0[01,00,00,0,,D2_tip96in384,1',
        'step-return2': '',
        'step-return3': '',
        'step-return4': ''
    }
)

HamiltonResponse(
    raw='{"command":"STAR-return","step-name":"gripPlace","step-return1":"0[01,00,00,0,,Waste_tip96in384,1","step-r
eturn2":"","step-return3":"","step-return4":"","id":"5c7e7aad-ef9c-42b8-9b9e-27837c7eb57b"}',
    status='SUCCESS',
    errors=[],
    data={
        'command': 'STAR-return',
        'step-name': 'gripPlace',
        'step-return1': '0[01,00,00,0,,Waste_tip96in384,1',
        'step-return2': '',
        'step-return3': '',
        'step-return4': ''
    }
)

HamiltonResponse(
    raw='{"command":"STAR-return","step-name":"channelTipEject","step-return1":"0[01,00,00,0,,,[02,00,00,0,,,[03,00
,00,0,0.0,COREGripTool_OnWaste_1000ul_0001,1[04,00,00,0,0.0,COREGripTool_OnWaste_1000ul_0001,2","step-return2":"","
step-return3":"","step-return4":"","id":"a5b432d2-039e-47d3-bd01-01a4085bb872"}',
    status='SUCCESS',
    errors=[],
    data={
        'command': 'STAR-return',
        'step-name': 'channelTipEject',
        'step-return1': 
'0[01,00,00,0,,,[02,00,00,0,,,[03,00,00,0,0.0,COREGripTool_OnWaste_1000ul_0001,1[04,00,00,0,0.0,COREGripTool_OnWast
e_1000ul_0001,2',
        'step-return2': '',
        'step-return3': '',
        'step-return4': ''
    }
)

2025-05-26 19:25:31,279 - adaptyv_lab.core.interface - INFO - Stopping Hamilton interface in gui mode
2025-05-26 19:25:32,010 - adaptyv_lab.core.interface - INFO - Resetting HSL file to default layout
2025-05-26 19:25:32,011 - adaptyv_lab.core.interface - INFO - Updating HSL file to use layout: STAR_OEM.lay
2025-05-26 19:25:32,015 - adaptyv_lab.core.interface - INFO - Hamilton interface stopped


In [ ]:
from src.hamilton_protocols.api.log_analyzer import (
    analyze_all_logs,
    get_combined_analysis,
)

log_dir = "C:\\Program Files (x86)\\HAMILTON\\LogFiles"

logs = analyze_all_logs(log_dir)
stats = get_combined_analysis(logs)
print(stats)